In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
import ipywidgets

In [2]:
def same(*args):
    for arg in args:
        assert args[0] == arg
    return args[0]

> **Note:**
> In cod, am folosit tabele indexate de la 0, insa in formule am folosit indexare de la 1. Deci $A^1$ se regaseste in `A[:,0]`, iar $\alpha_{0,0}$ se regaseste in `alpha[(-1,-1)]`

Rezolvam urmatoarea problema de optimizare, prin algoritmul simplex primal:

$$
(P)
\begin{cases}
f(x)=x_1+2x_2+2x_3-3x_4 \rightarrow \text{min} \\
x_1-2x_2-4x_3=6 \\
x_2+3x_3+x_4=8 \\
x_1,x_2,x_3,x_4\ge0 \\
\end{cases}
$$

Mai intai, extragem $c$, $b$, si $A$.

In [3]:
c = np.array([1, 2, 2, -3])
c

array([ 1,  2,  2, -3])

In [4]:
b = np.array([[6], [8]])
b

array([[6],
       [8]])

In [5]:
A = np.array([
    [1, -2, -4, 0],
    [0, 1, 3, 1],
])
A

array([[ 1, -2, -4,  0],
       [ 0,  1,  3,  1]])

In [ ]:
Js = [0, 3]

$A^1$ este prima baza, iar $A^4$ este cea de-a doua.

In [17]:
c = np.array([-4, -2, 0, 0, 0])
A = np.array([
    [1, 2, 1, 0, 0],
    [3, 0, 0, 1, 0],
    [0, 1, 0, 0, 1],
])
b = np.array([[6], [8], [4]])
Js = [2, 3, 4]

In [26]:
def generate_table(Is, Js, alpha):
    Is = list(Is)
    Js = list(Js)
    
    table = []
    for i in ["head"] + Is + [-1]:
        table.append("|")
        for j in ["head"] + Js + [-1]:
            match (i, j):
                case ("head", "head"):
                    table.append(" 1 ")
                case ("head", -1):
                    table.append(" X ")
                case (-1, "head"):
                    table.append(" b ")
                case ("head", val):
                    table.append(f" $A^{val+1}$ ")
                case (val, "head"):
                    table.append(f" $A^{val+1}$ ")
                case _:
                    table.append(f" $\\alpha_{{{i+1},{j+1}}}={alpha[(i,j)]}$ ")
            table.append("|")
    
        if i == "head":
            table.append("\n")
            table.append("|")
            for i in ["head"] + Js + [-1]:
                table.append("---|")
        
        table.append("\n")
    table = "".join(table)
    table = Markdown(table)
    return table

In [28]:
def simplex_primal(A, b, c, Js):
    same(2, len(A.shape), len(b.shape))
    same(1, len(c.shape))

    n = same(A.shape[1], c.shape[0])
    m = same(A.shape[0], b.shape[0])

    Js = set(Js)
    Is = set(range(n)) - Js

    # compute alpha
    alpha = dict()

    for i in Is:
        for j, val in zip(Js, A[:, i]):
            alpha[(i, j)] = val
    
    for j, val in zip(Js, b[:, 0]):
        alpha[(-1, j)] = val
        
    for i in Is:
        alpha[(i, -1)] = sum(
            alpha[(i, j)]*c[j]
            for j in Js
        ) - c[i]
        
    alpha[(-1, -1)] = sum(alpha[(-1, j)]*c[j] for j in Js)
        
    while True:
        display(generate_table(Is, Js, alpha))
        
        Isp = []
        for i in Is:
            if alpha[(i, -1)] > 0:
                Isp.append(i)

        # found optimal solution
        if len(Isp) == 0:
            return alpha[(-1, -1)]

        for i in Isp:
            fail = all(alpha[(i, k)] <= 0 for k in Js)
            if fail:
                raise RuntimeError("Function does not have lower bound, hence the problem has no solutions.")

        h = Isp[0]

        k = min(
            (j for j in Js if alpha[(h, j)] > 0),
            key=lambda j: alpha[(-1, j)] / alpha[(h, j)],
        )

        p = alpha[(h, k)]

        new_alpha = dict()
        for i in Is.union({-1}):
            for j in Js.union({-1}):
                match (i == h, j == k):
                    case (True, True):
                        new_alpha[(k, h)] = 1 / p
                    case (True, False):
                        new_alpha[(k, j)] = -(alpha[(i, j)] / p)
                    case (False, True):
                        new_alpha[(i, h)] = alpha[(i, j)] / p
                    case _:
                        new_alpha[(i, j)] = alpha[(i, j)] - alpha[(h, j)] * alpha[(i, k)] / p
        alpha = new_alpha
        
        Is.remove(h)
        Is.add(k)
        
        Js.remove(k)
        Js.add(h)


simplex_primal(A, b, c, Js)

| 1 | $A^3$ | $A^4$ | $A^5$ | X |
|---|---|---|---|---|
| $A^1$ | $\alpha_{1,3}=1$ | $\alpha_{1,4}=3$ | $\alpha_{1,5}=0$ | $\alpha_{1,0}=4$ |
| $A^2$ | $\alpha_{2,3}=2$ | $\alpha_{2,4}=0$ | $\alpha_{2,5}=1$ | $\alpha_{2,0}=2$ |
| b | $\alpha_{0,3}=6$ | $\alpha_{0,4}=8$ | $\alpha_{0,5}=4$ | $\alpha_{0,0}=0$ |


| 1 | $A^1$ | $A^3$ | $A^5$ | X |
|---|---|---|---|---|
| $A^2$ | $\alpha_{2,1}=0.0$ | $\alpha_{2,3}=2.0$ | $\alpha_{2,5}=1.0$ | $\alpha_{2,0}=2.0$ |
| $A^4$ | $\alpha_{4,1}=0.3333333333333333$ | $\alpha_{4,3}=-0.3333333333333333$ | $\alpha_{4,5}=-0.0$ | $\alpha_{4,0}=-1.3333333333333333$ |
| b | $\alpha_{0,1}=2.6666666666666665$ | $\alpha_{0,3}=3.3333333333333335$ | $\alpha_{0,5}=4.0$ | $\alpha_{0,0}=-10.666666666666666$ |


| 1 | $A^1$ | $A^2$ | $A^5$ | X |
|---|---|---|---|---|
| $A^3$ | $\alpha_{3,1}=-0.0$ | $\alpha_{3,2}=0.5$ | $\alpha_{3,5}=-0.5$ | $\alpha_{3,0}=-1.0$ |
| $A^4$ | $\alpha_{4,1}=0.3333333333333333$ | $\alpha_{4,2}=-0.16666666666666666$ | $\alpha_{4,5}=0.16666666666666666$ | $\alpha_{4,0}=-1.0$ |
| b | $\alpha_{0,1}=2.6666666666666665$ | $\alpha_{0,2}=1.6666666666666667$ | $\alpha_{0,5}=2.333333333333333$ | $\alpha_{0,0}=-14.0$ |


-14.0

$\alpha_{0,1}=6\ge0$ si $\alpha_{0,4}=8\ge0$ $\Rightarrow$ baza primal admisibila \
$\alpha_{2,0}=-1\le0$ si $\alpha_{3,0}=-15\le0$ $\Rightarrow$ baza primal admisibila \
$\Rightarrow$ $B$ este baza optima pentru problema $(P)$ \
$\Rightarrow$ valoarea minima a lui $f$ este $\alpha_{0,0}=-18$ \
obtinuta prin $x=(x_1,x_2,x_3,x_4)=(6,0,0,8)$

verificare:

In [10]:
def f(x):
    return np.sum(x * c)

In [11]:
f(np.array([6, 0, 0, 8]))

ValueError: operands could not be broadcast together with shapes (4,) (5,) 